<a href="https://colab.research.google.com/github/younga-Lee/younga-Lee.github.io/blob/master/crnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

## 데이터 준비

데이터 리스트 만들기

In [ ]:
import re
file_list = os.listdir('/content/drive/MyDrive/deep learning/train64')

tag=[]
for i in file_list:
  a = re.sub(r'[0-9]+','',i)
  a =  a.replace('.mp','')
  tag.append(a)

In [ ]:
train_df = pd.DataFrame()
train_tag =[]

train_df['video_name'] = file_list
for i in tag:
  train_tag.append(i.split(',')[0])
train_df['tag'] = train_tag
train_df.head()

,video_name,tag
0,"가시,날카롭다,뾰족하다,예리하다,예민하다10...",가시
1,"가시,날카롭다,뾰족하다,예리하다,예민하다10...",가시
2,"가시,날카롭다,뾰족하다,예리하다,예민하다10...",가시
3,"가시,날카롭다,뾰족하다,예리하다,예민하다10...",가시
4,"가시,날카롭다,뾰족하다,예리하다,예민하다10...",가시


In [ ]:
test_df = pd.DataFrame()

test_list = os.listdir('/content/drive/MyDrive/deep learning/test64')
test_df['video_name'] = test_list
test_tag=[]
for i in test_list:
  a = re.sub(r'[0-9]+','',i)
  a =  a.replace('.mp','')
  test_tag.append(a)
test_df['tag'] = test_tag

test_df.head()

,video_name,tag
0,개3.mp4,개
1,개8.mp4,개
2,곤충5.mp4,곤충
3,곤충9.mp4,곤충
4,꽃6.mp4,꽃


## 비디오 특성 확인

싱글 비디오

In [ ]:
cap =  cv2.VideoCapture('/content/drive/MyDrive/deep learning/train64/호랑이,범105.mp4')

# 비디오 프레임 크기, 전체 프레임수, FPS 등 출력
print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Frame count:', int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

fps = cap.get(cv2.CAP_PROP_FPS)
print('FPS:', fps)

Frame width: 64
Frame height: 48
Frame count: 149
FPS: 25.0


비디오 프레임 크기

In [ ]:
frame_count_list =[]
for i in file_list:
    vcap = cv2.VideoCapture('/content/drive/MyDrive/deep learning/train64/' + i)

    if vcap.isOpened(): 

        frame_count = vcap.get(cv2.CAP_PROP_FRAME_COUNT)
        frame_count = vcap.get(7)
        frame_count_list.append(frame_count)
msl = max(frame_count_list)
  

초매개변수 정의

In [ ]:
IMG_SIZE = 233 #이미지 크기 반으로 

BATCH_SIZE = 64
EPOCHS = 50

MAX_SEQ_LENGTH = int(msl)
NUM_FEATURES = 2048

In [ ]:
IMG_SIZE = 233 #이미지 크기 반으로 

BATCH_SIZE = 64
EPOCHS = 1000

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 2048

In [ ]:

#def crop_center_square(frame): pose부분 크롭하면 정확도가 향상되지않을까
#    y, x = frame.shape[0:2]
#    min_dim = min(y, x)
#    start_x = (x // 2) - (min_dim // 2)
#    start_y = (y // 2) - (min_dim // 2)
#    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


#비디오 로드
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read() #frame받아오기
            if not ret:
                break
            #frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            #frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
test = load_video('/content/drive/MyDrive/deep learning/test/개,강아지.mp4')

In [ ]:
test.shape

# model

CNN 전이학습 이용하여 feature 추출

1.  https://keras.io/api/applications/


In [ ]:
MAX_SEQ_LENGTH = 235
NUM_FEATURES = 512
IMG_SIZE = 244

EPOCHS = 50

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.VGG16(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.vgg16.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

58900480/58889256 [==============================] - 0s 0us/step


InceptionV3 정확도 33%

In [ ]:
def build_feature_extractor(): #InceptionV3
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

MobileNetV2 정확도 16%

In [ ]:
IMG_SIZE = 233 #이미지 크기 반으로 

BATCH_SIZE = 64
EPOCHS = 1000

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 1280

In [ ]:
def build_feature_extractor(): #MobileNetV2
    feature_extractor = keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

EfficientNetV2M 정확도 0..? 

In [ ]:
IMG_SIZE = 233 #이미지 크기 반으로 

BATCH_SIZE = 64
EPOCHS = 1000

MAX_SEQ_LENGTH = 200
NUM_FEATURES = 1280

In [ ]:
def build_feature_extractor(): #EfficientNetV2M
    feature_extractor = keras.applications.EfficientNetV2M(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
#신경망 짜기~ 

문자열 -> 정수 인코딩 layer

> 신경망은 문자열 값을 이해하지 못하므로 모델에 입력하기 전에 숫자형식으로 변환해야함






In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=1, vocabulary=np.unique(train_df["tag"])
)

label_processor.get_vocabulary()

['[UNK]',
 '가시',
 '개',
 '곤충',
 '꽃',
 '나무',
 '낙엽',
 '날다',
 '늑대',
 '당나귀',
 '동물',
 '멧돼지',
 '사슴',
 '소나무',
 '씨',
 '열매',
 '자라다',
 '잔디',
 '호랑이']

모델에 넣을 데이터 준비

In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features`길이 맞추기 위해서 0으로 패딩
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

   
    for idx, path in enumerate(video_paths):
        
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "/content/drive/MyDrive/deep learning/train64")
test_data, test_labels = prepare_all_videos(test_df, "/content/drive/MyDrive/deep learning/test64")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (167, 399, 2048)
Frame masks in train set: (167, 399)


RNN 신경망쌓기

In [ ]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(128, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(64)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


def run_experiment():
    #filepath = "/tmp/video_classifier"
    #checkpoint = keras.callbacks.ModelCheckpoint(
    #    filepath, save_weights_only=True, save_best_only=True, verbose=1
    #)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        #validation_split=0.3,
        epochs=EPOCHS,
        #callbacks=[checkpoint],
    )

    #seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

## 예측

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
